In [1]:
import numpy as np
import pandas as pd

from googletrans import Translator
from langid import classify

from nltk.corpus import stopwords

import bertopic

import sklearn

import re
import pymorphy2

import pickle

from typing import Union, Any

import yaml

import import_ipynb
from parse_1 import txt_file_to_list
from preprocessing_2 import translator_from_to, preprocess_text, \
    lemmatization_text, list_with_strings_preproc
from train_ml_4 import from_csv

import warnings
warnings.filterwarnings("ignore")

# Тестирование на новых данных
В этом ноутбуке подгружаю необработанные комментарии, которые модель еще не видела, и делаю на них предсказания

In [48]:
config_path = '../config/params.yml'
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

In [4]:
test_comments_path = config['nlp_modeling']['test_comments']

In [5]:
# Выгрузка комментариев
test_comments = txt_file_to_list(test_comments_path)

In [9]:
def from_pickle(path: str):
    """
    Загрузка данных по пути с формата pickle
    :params path: путь до файла
    :return: данне любого типа, которые были записаны в файле
    """
    with open(path, 'rb') as f:
        file = pickle.load(f)
    return file

In [10]:
embed_model_path = config['nlp_modeling']['embed_model']
pca_model_path = config['train_ml']['pca_model']
catboost_model_path = config['train_ml']['catboost_model']

In [11]:
# Подгружаю модели, которые будут участвовать в предсказаниях
embed_model = from_pickle(embed_model_path)
pca_model = from_pickle(pca_model_path)
catboost_model = from_pickle(catboost_model_path)

In [12]:
def get_embeddings(model: bertopic.backend._sentencetransformers.SentenceTransformerBackend, 
                   data: Union[list[str], str]) -> np.ndarray:
    """
    Получение ембеддингов из списка с текстами при помощи оученой модели 
    эмбеддингов
    :params model: экземпляр модели эмбеддингов из бертопика
    :params data: список текстов-документов
    :return: массив эмбеддингов
    """
    embeds = model.embed(data)
    return embeds

In [13]:
def compress_with_pca(model: sklearn.decomposition._pca.PCA,
                      data: np.ndarray) -> np.ndarray:
    """
    Сжатие колонок при помощи обученной модели pca
    :params model: экземпляр модели pca
    :params data: массив объектов признаков
    :return: сжатый массив объектов признаков
    """
    compressed_data = model.transform(data)
    return compressed_data

In [14]:
def ml_pred(model: Any, data: np.ndarray) -> (np.ndarray, np.ndarray):
    """
    предсказание на модели машинного обучения на входных данных
    :params model: экземпляр модели машинного обучения
    :params data: массив объектов признаков
    :return: вектор ответов
    :return: массив вероятностей определения к каждому из классов
    """
    preds = model.predict(data)
    preds_proba = model.predict_proba(data)
    return preds, preds_proba

In [193]:
def general_test_pipeline(data: Union[list[str], str], 
                          dest: str, 
                          list_with_strings_preproc,
                          preprocess_text: function,
                          lemmatization_text: function,
                          embed_model: bertopic.backend._sentencetransformers.SentenceTransformerBackend, 
                          pca_model: sklearn.decomposition._pca.PCA, 
                          ml_model: Any
                          ) -> (np.ndarray, np.ndarray):
    """
    Составление общего пайплайна для выдачи предсказаний из сырых комментариев
    :params data: список с необработанными документами
    :params dest: язык, на который нужно перевести документы
    :params list_with_strings_preproc: функция, которая очищает текст
    :params preprocess_text: функция, производящая очистку строк в списке и 
                             удаление стоп-слов 
                             (для фунции list_with_strings_preproc)
    :params lemmatization_text: функция, приводящая слова в строке в списке 
                                к начальной форме
                                (для функции list_with_strings_preproc)
    :params embed_model: экземпляр модели эмбеддингов из бертопика
    :params pca_model: экземпляр модели pca
    :params ml_model: экземпляр модели машинного обучения
    :return: вектор ответов
    :return: массив вероятностей определения к каждому из классов
    """
    if not isinstance(data, list):
        if not isinstance(data, str):
            raise TypeError('Введите строку или список из строк')
        data = [data]

    data = translator_from_to(data, dest)

    data = list_with_strings_preproc(data, 
                                     preprocess_text, 
                                     lemmatization_text)

    data = get_embeddings(embed_model, data)
    data = compress_with_pca(pca_model, data)
    preds, preds_proba = ml_pred(ml_model, data)

    preds_proba = preds_proba.ravel()
    

    return preds, preds_proba

In [16]:
dest_lang = config['preprocessing']['dest']

In [17]:
preds, preds_proba = general_test_pipeline(test_comments, dest_lang,
                                           list_with_strings_preproc,
                                           preprocess_text, lemmatization_text,
                                           embed_model, pca_model, 
                                           catboost_model)

In [211]:
preds

array([[ 1],
       [30],
       [ 1],
       ...,
       [ 0],
       [ 0],
       [ 0]], dtype=int64)

Создание функции, которая будет показывать, к каким топ n топикам относится комментарий

In [49]:
topics_info_path = config['nlp_modeling']['topics_info']
top_n_topics = config['evaluate']['top_n_topics']

In [22]:
topics_info = from_csv(topics_info_path)

In [149]:
def get_top_n_topics(list_probas: np.ndarray, 
                     top_n_topics: int, 
                     topics_info_full: pd.DataFrame) -> pd.DataFrame:
    """
    Вывод топ n топиков по вектору вероятностей отнесения к классам 
    у комментария
    :params list_probas: вектор вероятностей отнесения к классам
    :params top_n_topics: количество самых топовых по вероятностям топиков
    :params topics_info_full: датафрейм с метками топиков и их пояснением
    """
    # Отсортированный список вероятностей 
    top_probas = list(map(lambda x: round(x * 100, 2), 
                          np.sort(list_probas)[::-1][:top_n_topics]))
    
    # Отсортированный по вероятностям список меток класса
    top_topics = np.argsort(list_probas)[::-1][:top_n_topics]
    
    # Сопоставление меток класса с их описанием
    top_topics_names = []
    for i in top_topics:
        topic_name = topics_info_full[topics_info_full['Topic'] == i].values[0][2]
        top_topics_names.append(topic_name)

    top_df = pd.DataFrame({
        'Probability': top_probas,
        'Topic_name': top_topics_names
    })
    
    return top_df

In [155]:
test_comments[33]

'гитара обучаться играть программа обучение нота учить'

In [154]:
get_top_n_topics(preds_proba[33], top_n_topics, topics_info)

,Probability,Topic_name
0,96.49,Оценка мастерства гитаристов
1,1.01,Соединение барабанщика и гитариста
2,0.60,Учитель и ученик в музыке
3,0.48,Мусор
4,0.18,Выразительный бас


In [197]:
# Проверим како-нибудь другой комментарий
_, proba_new = general_test_pipeline("Круто сыграл, бас бочка звучит мощно", 
                                     dest_lang, list_with_strings_preproc,
                                     preprocess_text, lemmatization_text,
                                     embed_model, pca_model, catboost_model)

In [198]:
get_top_n_topics(proba_new, top_n_topics, topics_info)

,Probability,Topic_name
0,16.78,Выразительный бас
1,16.75,Эволюция музыкальных предпочтений
2,13.68,Шумное и громкое видео
3,12.55,Мусор
4,10.95,Восхищение игрой на барабанах


In [199]:
# Несоответствие типов данных
general_test_pipeline(5, dest_lang, list_with_strings_preproc,preprocess_text, 
                      lemmatization_text, embed_model, pca_model, 
                      catboost_model)

TypeError: Введите строку или список из строк